In [1]:
# DELETE BEFORE PUBLISHING
# This is just here so you can preview the styling on your local machine

from IPython.core.display import HTML
HTML("""
<style>
.usecase-title, .usecase-duration, .usecase-section-header {
    padding-left: 15px;
    padding-bottom: 10px;
    padding-top: 10px;
    padding-right: 15px;
    background-color: #0f9295;
    color: #fff;
}

.usecase-title {
    font-size: 1.7em;
    font-weight: bold;
}

.usecase-authors, .usecase-level, .usecase-skill {
    padding-left: 15px;
    padding-bottom: 7px;
    padding-top: 7px;
    background-color: #baeaeb;
    font-size: 1.4em;
    color: #121212;
}

.usecase-level-skill  {
    display: flex;
}

.usecase-level, .usecase-skill {
    width: 50%;
}

.usecase-duration, .usecase-skill {
    text-align: right;
    padding-right: 15px;
    padding-bottom: 8px;
    font-size: 1.4em;
}

.usecase-section-header {
    font-weight: bold;
    font-size: 1.5em;
}

.usecase-subsection-header, .usecase-subsection-blurb {
    font-weight: bold;
    font-size: 1.2em;
    color: #121212;
}

.usecase-subsection-blurb {
    font-size: 1em;
    font-style: italic;
}
</style>
""")

<div class="usecase-title">New Business Location Use Case</div>

<div class="usecase-authors"><b>Authored by: </b> Steven Tuften</div>

<div class="usecase-duration"><b>Duration:</b> 90 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<div class="usecase-section-header">Scenario</div>

#### As a Cafe, Restaurant or Bar, I am looking for commercial space in the City of Melbourne where I can open a new venue or extend my existing venue.

#### I would like to know where similar businesses are located and the density of residents and office workers in comparison.

#### I want to know the number of seats I should provide based on seating capacity at other similar establishments in the same area.

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- understand what CLUE data is and how to access it
- have explored a dataset derived from the CLUE survey
- learnt how to visualise CLUE data using different mapping visualisation techniques

<div class="usecase-section-header">A brief introduction to CLUE data</div>

The City of Melbourne conducts a comprehensive bi-annual survey of its residents and businesses called the "Census of Land Use and Employment (CLUE)". CLUE captures key information on land use, employment, and economic activity across the City of Melbourne.

CLUE datasets are a valuable tool for businesses looking to invest in the City of Melbourne and for researchers wanting to understand those factors that influence and shape the social and economic dynamics of Australia's second largest metropolis and one of the world's most liveable cities.

CLUE data assists the City of Melbourne's business planning, policy development and strategic decision making. Investors, consultants, students, urban researchers, property analysts, businesses and developers can take advantage of CLUE to understand customers, the marketplace and the changing form and nature of the city.

Source: __[CLUE]( https://data.melbourne.vic.gov.au/stories/s/CLUE/rt3z-vy3t?src=hdr)__

This use case utilises various CLUE datasets to illustrate their value to Data Scientists, Researchers and Software Developers. 

### CLUE Geospatial Data

CLUE Data is often coded to a specific location (Latitude and Longitude) and/or to a City precinct, referred to as the "CLUE small area". Datasets may also include the individual city block within a precinct referred to by its CLUE Block ID.

The geospatial coordinates describing these areas as polygons can be downloaded in GeoJSON format and used to show shaded areas on a map, known as a choropleth map. This can be a useful technique for illustrating broad trends or statistics for a city area rather than a specific location.

A map visualisation of CLUE Blocks and small areas can be found at the following links:
- __[CLUE small areas](https://data.melbourne.vic.gov.au/Business/Small-Areas-for-Census-of-Land-Use-and-Employment-/gei8-3w86)__
- __[CLUE Blocks](https://data.melbourne.vic.gov.au/Business/Blocks-for-Census-of-Land-Use-and-Employment-CLUE-/aia8-ryiq)__ 


<div class="usecase-section-header">Which CLUE data should I use?</div>

To begin we shall first import the necessary libraries to support our exploratory data analysis and visualisation of the CLUE data.

The following are core packages required for this exercise:

- The sodapy package is required specifically for accessing open data from SOCRATA compliant open data web sites.

- The plotly.express package lets use build interact maps using map box services.

In [1]:
import os
import time

from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import requests
import cred


In [6]:
#base_url
BASE_URL = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'

domain = 'data.melbourne.vic.gov.au'

#Dataset name
Res_Dataset = 'residential-dwellings'
Cafe_Dataset = "cafes-and-restaurants-with-seating-capacity"
NUMBER_OF_RECORDS = 100

# Add a filter for the number of results returned
FILTERS = 'records?limit={0}&offset=0&timezone=UTC'.format(NUMBER_OF_RECORDS)
# Same as
FILTERS = f'records?limit={NUMBER_OF_RECORDS}&offset=0&timezone=UTC'

In [7]:
Res_URL = f'{BASE_URL}{Res_Dataset}/{FILTERS}'

In [8]:
def get_data(base, data_url, size = 0):
    
    filters = f'records?limit={100}&offset={size}&timezone=UTC'
    url = f'{base}{data_url}/{filters}&apikey={cred.api_key}'
    result = requests.get(url)
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)
    
    # fix columns names
    records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()
    
    # Replace location.lat & location.lon
    column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='location' else a for a in column_names]
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    next_url = None
    
    # get next url
    if records_df.shape[0] != max_results: #added second cap to loop for truncation issue
        for l in links:
            if l['rel']=='next':
                next_url = l['href']+f'&apikey={cred.api_key}'
                
    return [records_df, next_url, column_names]

In [9]:
#call the function 
df_new, url_next, column_names = get_data(BASE_URL,Res_Dataset)

In [10]:
df_new

,id,timestamp,size,census_year,block_id,property_id,base_property_id,building_address,clue_small_area,dwelling_type,dwelling_number,longitude,latitude,location_lon,location_lat
0,e5a67a115fc82caef3255b6c0255e17a6925b2b9,2022-12-13T12:31:42.85Z,192,2020,333,106072,106072,9-11 Little Provost Street NORTH MELBOURNE VIC...,North Melbourne,House/Townhouse,1,144.94776,-37.80246,144.94776,-37.80246
1,e19d9dd31c4083c28c8777412ad3fae89d1b0bd0,2022-12-13T12:31:42.85Z,183,2020,333,107744,107744,37 Provost Street NORTH MELBOURNE VIC 3051,North Melbourne,House/Townhouse,1,144.94665,-37.80215,144.94665,-37.80215
2,d52758a5773dcacc6a5ce036220746ed8d9237af,2022-12-13T12:31:42.85Z,181,2020,333,107745,107745,39 Provost Street NORTH MELBOURNE VIC 3051,North Melbourne,House/Townhouse,1,144.94660,-37.80214,144.94660,-37.80214
3,1e158ed1f589e84cf46ca272dc83aa9e8d098201,2022-12-13T12:31:42.85Z,188,2020,333,107972,107972,612-616 Queensberry Street NORTH MELBOURNE VIC...,North Melbourne,House/Townhouse,8,144.94700,-37.80251,144.94700,-37.80251
4,12bbe6e0838146785a71fc000aee47d558c876cc,2022-12-13T12:31:42.85Z,192,2020,333,107979,107979,600-602 Queensberry Street NORTH MELBOURNE VIC...,North Melbourne,House/Townhouse,1,144.94738,-37.80255,144.94738,-37.80255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7d1153584c48e27e50b94fa6c95fa0f83bf119e5,2022-12-13T12:31:42.85Z,189,2020,344,109776,109776,602-604 Victoria Street NORTH MELBOURNE VIC 3051,North Melbourne,House/Townhouse,1,144.94623,-37.80445,144.94623,-37.80445
96,69654ceb3fe29f6405beaefc18aec6b17b946fbd,2022-12-13T12:31:42.85Z,189,2020,344,109780,109780,590-592 Victoria Street NORTH MELBOURNE VIC 3051,North Melbourne,House/Townhouse,1,144.94658,-37.80449,144.94658,-37.80449
97,1d24fc7ca55cb3e88a38c313d4b41a75582d76d9,2022-12-13T12:31:42.85Z,183,2020,344,109784,109784,580 Victoria Street NORTH MELBOURNE VIC 3051,North Melbourne,House/Townhouse,1,144.94690,-37.80452,144.94690,-37.80452
98,c5018fa93664fd9591b473418d2f42b27ac9b258,2022-12-13T12:31:42.85Z,184,2020,345,100590,100590,28-30 Atkin Street NORTH MELBOURNE VIC 3051,North Melbourne,House/Townhouse,2,144.94896,-37.80337,144.94896,-37.80337


In [9]:
column_names

['id',
 'timestamp',
 'size',
 'census_year',
 'block_id',
 'property_id',
 'base_property_id',
 'building_address',
 'clue_small_area',
 'dwelling_type',
 'dwelling_number',
 'longitude',
 'latitude',
 'location_lon',
 'location_lat']

In [16]:
# Create new EMPTy dataframe with columns
New_Res_Df = pd.DataFrame(columns=column_names)
New_Res_Df

,id,timestamp,size,census_year,block_id,property_id,base_property_id,building_address,clue_small_area,dwelling_type,dwelling_number,longitude,latitude,location_lon,location_lat


In [11]:
# Create function to add API output to dataframe
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

In [17]:
# Add initial output to datframe
New_Res_Df = add_to_dataframe(New_Res_Df, df_new)

In [13]:
New_Res_Df.shape

(100, 15)

In [18]:
# Get the rest of the Dataset
while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL,Res_Dataset, New_Res_Df.shape[0])
    New_Res_Df = add_to_dataframe(New_Res_Df, df_new)
    print(New_Res_Df.shape)
    if New_Res_Df.shape[0]>=9900: # Added for truncation issue
        break

(200, 15)
(300, 15)
(400, 15)
(500, 15)
(600, 15)
(700, 15)
(800, 15)
(900, 15)
(1000, 15)
(1100, 15)
(1200, 15)
(1300, 15)
(1400, 15)
(1500, 15)
(1600, 15)
(1700, 15)
(1800, 15)
(1900, 15)
(2000, 15)
(2100, 15)
(2200, 15)
(2300, 15)
(2400, 15)
(2500, 15)
(2600, 15)
(2700, 15)
(2800, 15)
(2900, 15)
(3000, 15)
(3100, 15)
(3200, 15)
(3300, 15)
(3400, 16)
(3500, 16)
(3600, 16)
(3700, 16)
(3800, 16)
(3900, 16)
(4000, 16)
(4100, 16)
(4200, 16)
(4300, 16)
(4400, 16)
(4500, 16)
(4600, 16)
(4700, 16)
(4800, 16)
(4900, 16)
(5000, 16)
(5100, 16)
(5200, 16)
(5300, 16)
(5400, 16)
(5500, 16)
(5600, 16)
(5700, 16)
(5800, 16)
(5900, 16)
(6000, 16)
(6100, 16)
(6200, 16)
(6300, 16)
(6400, 16)
(6500, 16)
(6600, 16)
(6700, 16)
(6800, 16)
(6900, 16)
(7000, 16)
(7100, 16)
(7200, 16)
(7300, 16)
(7400, 16)
(7500, 16)
(7600, 16)
(7700, 16)
(7800, 16)
(7900, 16)
(8000, 16)
(8100, 16)
(8200, 16)
(8300, 16)
(8400, 16)
(8500, 16)
(8600, 16)
(8700, 16)
(8800, 16)
(8900, 16)
(9000, 16)
(9100, 16)
(9200, 16)
(9300, 

In [20]:
#rename the columns to match the columns named by original coder Steven Tuften. As well, to change the dataset to match the column order
New_Res_Df.rename(columns={'property_id': 'pbs_property_id', 'base_property_id': 'bps_base_id',"building_address":"street_name","location_lon":"x_coordinate","location_lat":"y_coordinate"}, inplace=True)
columns_list = ["census_year","block_id","pbs_property_id","bps_base_id","street_name","clue_small_area","dwelling_type","dwelling_number","x_coordinate","y_coordinate"]
New_Res_Df = New_Res_Df[columns_list]

Next, we will look at one of the CLUE datasets to better understand its structure and how we can use it.

Our data requirements from this use case include the following:
- Number of Residential Dwellings per CLUE Block
- Number of Employees per CLUE Block
- Number of Seats (Indoor and Outdoor) per Venue and CLUE Block

For this exercise, we shall start by examining the Residential Dwelling dataset.
Each dataset in the Melbourne Open Data Portal has a unique identifier which can be used to retrieve the dataset using the sodapy library.

The Residential Dwelling dataset unique identifier is **rm92-h5tq**.
We will pass this identifier into the sodapy command below to retrieve this data.

This dataset is placed in a Pandas dataframe and we will inspect the first three rows.

In [21]:
# Retrieve the "CLUE Residential Dwellings 2020" dataset

print(f'The shape of dataset is {New_Res_Df.shape}.')
print('Below are the first few rows of this dataset:')

# Transpose the DataFrame for easier visual comparison. 
New_Res_Df.head(3).T

The shape of dataset is (9900, 10).
Below are the first few rows of this dataset:


,0,1,2
census_year,2018,2018,2018
block_id,377,377,377
pbs_property_id,596469,596470,623920
bps_base_id,596469,596470,623920
street_name,1C Hotham Place NORTH MELBOURNE 3051,1D Hotham Place NORTH MELBOURNE 3051,5 Hotham Place NORTH MELBOURNE 3051
clue_small_area,North Melbourne,North Melbourne,North Melbourne
dwelling_type,House/Townhouse,House/Townhouse,House/Townhouse
dwelling_number,1,1,1
x_coordinate,144.95288,144.95292,144.953
y_coordinate,-37.80073,-37.8007,-37.80057


We can see that there are 10,402 records and 10 fields describing each record.

Each record show us the number of dwellings for each individual property and the type of dwelling e.g. House/Townhouse, Residential Apartments, etc.

The location of each property is given using:
- Latitude and Longitude
- CLUE Small Area and Block ID
- Property Id

The Census year that the data was collected is also shown.

For our analysis of this dataset and others we will be restricting our analysis to the 2020 CLUE Census and summarising the data to CLUE Block level.

<div class="usecase-section-header">Summarising Residential Dwelling data</div>

We want to plot the density of both residential dwellings and employment at city block level rather than a specific property or address. We can use a __[choropleth map](https://en.wikipedia.org/wiki/Choropleth_map)__ to do this.

Let's start by summarising the data at CLUE small area and Block level.

*Note: We include CLUE Small Area as one of our group by fields so we can display the CLUE Small area name in the popup window when you hover over the area on the map.*

We want to summarise the data by summing the number of dwellings across all rows in the same CLUE Block.

The following cell creates a dataframe containing this summary of residential dwellings.

In [22]:
# Cast datatypes to correct type so we can summarise
New_Res_Df[['census_year', 'dwelling_number']] = New_Res_Df[['census_year', 'dwelling_number']].astype(int)
New_Res_Df[['x_coordinate', 'y_coordinate']] = New_Res_Df[['longitude', 'latitude']].astype(float)
New_Res_Df = New_Res_Df.convert_dtypes() # convert remaining to string
New_Res_Df.dtypes

# create the aggregate dataset
groupbyfields = ['block_id','clue_small_area']
aggregatebyfields = {'dwelling_number': ["sum"]}

dwellingsByBlock = pd.DataFrame(df_new.groupby(groupbyfields, as_index=False).agg(aggregatebyfields))

# Dataframse Group by creates two levels of headings
# so we flatten the headings to make it easier to extract data for plotting
dwellingsByBlock.columns = dwellingsByBlock.columns.map(''.join) # flatten column header
dwellingsByBlock.rename(columns={'clue_small_area': 'clue_area'}, inplace=True) #rename to match GeoJSON extract
dwellingsByBlock.rename(columns={'dwelling_numbersum': 'dwelling_count'}, inplace=True)
dwellingsByBlock.head(5)

KeyError: "None of [Index(['longitude', 'latitude'], dtype='object')] are in the [columns]"

<div class="usecase-section-header">Visualising Residential Dwelling on a Choropleth Map</div>

We use the __[Plotly Python Open Source Graphing Library](https://plotly.com/python/)__ to generate maps from __[mapbox](https://www.mapbox.com/)__.

Creating a choropleth map requires us to know the geometry(shape) of each CLUE Block area as a collection of latitude and longitude points defining a polygon. This data can be downloaded from the Melbourne Open Data Portal in __[GeoJSON](https://en.wikipedia.org/wiki/GeoJSON)__ format.

We also need to supply the data to be used to highlight the CLUE Blocks and that data must include the same unique identifier for each Block contained in the GeoJSON data set.

Below we extract the Melbourne CLUE Block polygons into a JSON datatype.

**The final line in the cell displays the unique key for each polygon which must also exist in the Residential Dwelling dataset.**

In [27]:
import json
import geopandas as gpd

block = gpd.read_file('https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/blocks-for-census-of-land-use-and-employment-clue/exports/geojson') 
#block["features"][0]['properties'].keys()
block

,geo_point_2d,block_id,clue_area,geometry
0,"{'lon': 144.95049282288122, 'lat': -37.8229616...",1112,Docklands,"POLYGON ((144.94792 -37.82337, 144.94809 -37.8..."
1,"{'lon': 144.94085920366408, 'lat': -37.7853742...",927,Parkville,"POLYGON ((144.94262 -37.78663, 144.94250 -37.7..."
2,"{'lon': 144.94600024715058, 'lat': -37.7776873...",929,Parkville,"POLYGON ((144.94259 -37.77872, 144.94436 -37.7..."
3,"{'lon': 144.94361235073427, 'lat': -37.7967014...",318,North Melbourne,"POLYGON ((144.94472 -37.79613, 144.94177 -37.7..."
4,"{'lon': 144.94371829763847, 'lat': -37.7929397...",302,North Melbourne,"POLYGON ((144.94539 -37.79253, 144.94229 -37.7..."
...,...,...,...,...
601,"{'lon': 144.92713678681392, 'lat': -37.7891757...",2509,Kensington,"POLYGON ((144.92715 -37.78854, 144.92642 -37.7..."
602,"{'lon': 144.92314081754333, 'lat': -37.7880574...",2501,Kensington,"POLYGON ((144.92131 -37.78746, 144.92165 -37.7..."
603,"{'lon': 144.9413466509413, 'lat': -37.79011317...",2384,North Melbourne,"POLYGON ((144.94330 -37.79029, 144.94237 -37.7..."
604,"{'lon': 144.9413608525311, 'lat': -37.79317032...",2389,North Melbourne,"POLYGON ((144.94074 -37.79235, 144.94050 -37.7..."


Now using just one function call called 'choropleth_mapbox' we can display an interactive map using the **block** GeoJSON data to define the regions and the **dwellingsByBlock** dataframe to define the summarised data by block.

In [26]:
# Display the choropleth map
fig = px.choropleth_mapbox(dwellingsByBlock, # pass in the summarised dwellings per block
                           geojson=block, # pass in the GeoJSON data defining the CLUE Block polygons
                           locations='block_id', # define the unique identifier for the Blocks from the dataframe
                           color='dwelling_count', # change the colour of the block region according to the dwelling count
                           color_continuous_scale=["#FFFF88", "yellow", "orange", "orange",
                                                   "orange", "darkorange", "red", "darkred"], # define custom colour scale
                           range_color=(0, dwellingsByBlock['dwelling_count'].max()), # set the numeric range for the colour scale
                           featureidkey="properties.block_id", # define the Unique polygon identifier from the GeoJSON data
                           mapbox_style="stamen-toner", # set the visual style of the map
                           zoom=12.15, # set the zoom level
                           center = {"lat": -37.813, "lon": 144.945}, # set the map centre coordinates
                           opacity=0.5, # opacity of the choropleth polygons
                           hover_name='clue_area', # the title of the hover pop up box
                           hover_data={'block_id':True,'dwelling_count':True}, # defines which dataframe fields to display
                                                                               # in the hover popup box
                           labels={'dwelling_count':'Number of Dwellings','block_id':'CLUE Block Id'}, # defines labels for
                                                                               # the hover popup box
                           title='Residential Dwellings by CLUE Block Id for 2020', # Title for plot
                           width=950, height=800 # dimensions of plot in pixels
                          )
fig.show()

NameError: name 'dwellingsByBlock' is not defined

You've successfully used Melbourne CLUE Open Data and Plotly to visualise residential density in the City of Melbourne!<br>
Now zoom in and out on the map above to explore the city and areas of high and low residential density.<br><br>
This is your first step to selecting a suitable location for your new business!

__[You can explore the Residential Density data here](../dataanalysis/eda-clue-residentialdwellings.ipynb)__.

<div class="usecase-section-header">Visualising Residential Density and Cafe or Restaurant Seating</div>

To build our view of cafe venue seating and how it relates to residential density we need to visualise both datasets on the same interactive map view.

We can do this by adding a new layer (or "trace" as it is called in Plotly) to our previous map of residential density.

Let's extract the Melbourne CLUE cafe, restaurant, bistro seats dataset and summarise it so its ready to plot.

In [13]:
#call the function cafe dataset 
df_new, url_next, column_names = get_data(BASE_URL,Cafe_Dataset)
# Create new EMPTy dataframe with columns
New_Cafe_Df = pd.DataFrame(columns=column_names)
New_Cafe_Df

# Add initial output to datframe
New_Cafe_Df = add_to_dataframe(New_Cafe_Df, df_new)

# Get the rest of the Dataset
while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL,Res_Dataset, New_Cafe_Df.shape[0])
    New_Cafe_Df = add_to_dataframe(New_Cafe_Df, df_new)
    print(New_Cafe_Df.shape)
    if New_Cafe_Df.shape[0]>=9900: # Added for truncation issue
        break


(200, 22)
(300, 22)
(400, 22)
(500, 22)
(600, 22)
(700, 22)
(800, 22)
(900, 22)
(1000, 22)
(1100, 22)
(1200, 22)
(1300, 22)
(1400, 22)
(1500, 22)
(1600, 22)
(1700, 22)
(1800, 22)
(1900, 22)
(2000, 22)
(2100, 22)
(2200, 22)
(2300, 22)
(2400, 22)
(2500, 22)
(2600, 22)
(2700, 22)
(2800, 22)
(2900, 22)
(3000, 22)
(3100, 22)
(3200, 22)
(3300, 22)
(3400, 22)
(3500, 22)
(3600, 22)
(3700, 22)
(3800, 22)
(3900, 22)
(4000, 22)
(4100, 22)
(4200, 22)
(4300, 22)
(4400, 22)
(4500, 22)
(4600, 22)
(4700, 22)
(4800, 22)
(4900, 22)
(5000, 22)
(5100, 22)
(5200, 22)
(5300, 22)
(5400, 22)
(5500, 22)
(5600, 22)
(5700, 22)
(5800, 22)
(5900, 22)
(6000, 22)
(6100, 22)
(6200, 22)
(6300, 22)
(6400, 22)
(6500, 22)
(6600, 22)
(6700, 22)
(6800, 22)
(6900, 22)
(7000, 22)
(7100, 22)
(7200, 22)
(7300, 22)
(7400, 22)
(7500, 22)
(7600, 22)
(7700, 22)
(7800, 22)
(7900, 22)
(8000, 22)
(8100, 22)
(8200, 22)
(8300, 22)
(8400, 22)
(8500, 22)
(8600, 22)
(8700, 22)
(8800, 22)
(8900, 22)
(9000, 22)
(9100, 22)
(9200, 22)
(9300, 

In [15]:
# Pull dataset from Melbourne Open Data Portal
data_dyqx_cfn5 = New_Cafe_Df # Melbourne CLUE Cafe, restaurant, bistro seats

# Cast columns to correct data type
integer_columns = ['census_year', 'block_id', 'property_id', 'base_property_id', 'industry_anzsic4_code', 'number_of_seats']
fp_columns = ['x_coordinate', 'y_coordinate']
data_dyqx_cfn5[integer_columns] = data_dyqx_cfn5[integer_columns].astype(int)
data_dyqx_cfn5[fp_columns] = data_dyqx_cfn5[fp_columns].astype(float)
data_dyqx_cfn5 = data_dyqx_cfn5.convert_dtypes() # convert remaining to string

# Summarise venue seating by location
groupbyfields = ['clue_small_area','block_id','y_coordinate','x_coordinate']
aggregatebyfields = {'number_of_seats': ["sum"]}

seatsByLocn = pd.DataFrame(data_dyqx_cfn5.groupby(groupbyfields, as_index=False).agg(aggregatebyfields))
seatsByLocn.columns = seatsByLocn.columns.map(''.join) # flatten column header
seatsByLocn.rename(columns={'clue_small_area': 'clue_area'}, inplace=True) #rename to match GeoJSON extract
seatsByLocn.rename(columns={'number_of_seatssum': 'number_of_seats'}, inplace=True) #rename to match GeoJSON extract
seatsByLocn['number_of_seats'] = seatsByLocn['number_of_seats'].astype(int)

# Calculate scale for drawing each bubble on scatter map plot
all_data_diffq = (seatsByLocn["number_of_seats"].max() - seatsByLocn["number_of_seats"].min()) / 16
seatsByLocn['scale'] = (seatsByLocn["number_of_seats"] - seatsByLocn["number_of_seats"].min()) / all_data_diffq + 1
seatsByLocn['scale'] = seatsByLocn['scale'].astype(int)+2
seatsByLocn.head(10)

ValueError: cannot convert float NaN to integer

Above we can see our summary dataframe has calculated the total number of seats (indoor and outdoor) at each unique locations (latitude and longitude).

Since there is such a wide variance in venue seating across the city we need to scale the size of the bubbles drawn on the map to just a few (16) distinct sizes.

We set the lowest scale to 3 to ensure even the smallest venue's bubble is large enough when one zooms in at block level.

The next step is to display both the Choropleth and Scatter maps.
We first draw the choropleth map showing residential density.
We then draw the scatter plot assigning it as a trace (aka "layer") to the existing figure then show both.

In [ ]:
# Plot residential density and venue seating
fig = px.choropleth_mapbox(dwellingsByBlock, geojson=block, locations='block_id', color='dwelling_count',
                           color_continuous_scale=["#FFFF88", "yellow", "orange", "orange",
                                                   "orange", "darkorange", "red", "darkred"],
                           range_color=(0, dwellingsByBlock['dwelling_count'].max()),
                           featureidkey="properties.block_id",
                           mapbox_style="stamen-toner", #"carto-positron",
                           zoom=12.15,
                           center = {"lat": -37.813, "lon": 144.945},
                           opacity=0.5,
                           hover_name='clue_area',
                           hover_data={'block_id':True,'dwelling_count':True},
                           labels={'dwelling_count':'Number of Dwellings','block_id':'CLUE Block Id'},
                           title='Residential Dwellings Density & Venue Seating (2020)',
                           width=950, height=800
                          )

# Plot of venue seating
fig2 = px.scatter_mapbox(seatsByLocn, lat="y_coordinate", lon="x_coordinate", size="scale",
                        mapbox_style="stamen-toner",
                        zoom=12.15,
                        center = {"lat": -37.813, "lon": 144.945},
                        opacity=0.70,
                        hover_name="clue_area",
                        hover_data={"block_id":True,"scale":False,"number_of_seats":True,"x_coordinate":False,"y_coordinate":False},
                        color_discrete_sequence=['purple'],
                        labels={'number_of_seats':'Number of Seats', 'block_id':'CLUE Block Id'},
                        width=950, height=800)
fig.add_trace(fig2.data[0])
fig.update_geos(fitbounds="locations", visible=False)

fig.show()

You've successfully used Melbourne CLUE Open Data and Plotly to visualise residential density and venue seating in the City of Melbourne in one map!<br>
Now zoom in and out on the map above to explore the city and areas of high residential density but low venue seating.<br><br>
This could be a possible location for your new business!

__[You can explore the Venue Seating data in more detail here](../dataanalysis/eda-clue-venueseats.ipynb)__.

<div class="usecase-section-header">Building an Interactive Visualisation for New Business Location</div>

In the previous step we saw how we can create a new layer, also called a trace, to an existing mapbox plot in order to visualise both residential density and cafe or Restaurant venue seating on the one map.

We now wish to add Employment Density to this visualisation.
Since Employment density and Residential density both require use a choropleth map to visualise data at CLUE block level, we cannot overlay these two layers at the same time.

We therefore need a way to select the base choropleth map to show either residential density or employment density and then optionally turn on or off the venue seating as an additional scatter map box layer.

To achieve this interactivity we can make use of Plotly express functions to build a drop down menu and button to be overlaid on the map. 

We will require three datasets and associated layers (traces) for this visualisation.

Let's start by extracting our third dataset titled __["Employment per industry for blocks 2020"](https://data.melbourne.vic.gov.au/Business/Employment-per-industry-for-blocks-2020/qnju-it8g)__ and performing some data preparation prior to plotting.

*Note: The ***"Employment per industry for blocks 2020"*** dataset is a summary of employment at CLUE Block level and so we do not need to perform a groupby aggregation on the dataset.*

In [ ]:
# Pull dataset from Melbourne Open Data Portal
data_qnju_it8g = pd.DataFrame.from_dict(client.get_all('qnju-it8g')) # Employment per industry for blocks 2020

# Filter out unwanted columns
columnsToKeep = ['clue_small_area','block_id','total_employment_in_block']
employmentByBlock = data_qnju_it8g.filter(columnsToKeep)

# Rename to match GeoJSON extract
employmentByBlock.rename(columns={'clue_small_area': 'clue_area'}, inplace=True)

# Replace all NaNs with zero
employmentByBlock.fillna(value=0,inplace=True)

# Cast columns to correct datatype
employmentByBlock[['block_id','total_employment_in_block']] = employmentByBlock[['block_id','total_employment_in_block']].astype(int)
employmentByBlock = employmentByBlock.convert_dtypes() # convert remaining to string

# Exclude summary total for all of City of Melbourne
employmentByBlock = employmentByBlock[employmentByBlock['block_id'] > 0] 

# Display sample data
employmentByBlock.head(5)

Now we have a dataset showing total number of employees by CLUE block, let's visualise it as a choropleth map and overlay venue seating.

In this map visualisation we will use a different map style called "open-street-map" which lets us identify the names of venues close to where the venue seating measures have been reported. **Note that not all venues may have been marked on Open Street Maps.**

Mapbox styles which do not require a Mapbox API token are 'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'. Mapbox styles which do require a Mapbox API token are 'basic', 'streets', 'outdoors', 'light', 'dark', 'satellite', 'satellite- streets'.

**Source:** __[plotly.express.line_mapbox documentation](https://plotly.com/python-api-reference/generated/plotly.express.line_mapbox.html)__

In [ ]:
# Plot employment density
fig = px.choropleth_mapbox(employmentByBlock, geojson=block, locations='block_id', color='total_employment_in_block',
                           color_continuous_scale="Blues",
                           range_color=(0, employmentByBlock['total_employment_in_block'].max()),
                           featureidkey="properties.block_id",
                           mapbox_style="open-street-map",
                           zoom=12.15,
                           center = {"lat": -37.813, "lon": 144.945},
                           opacity=0.5,
                           hover_name='clue_area',
                           hover_data={'block_id':True,'total_employment_in_block':True},
                           labels={'total_employment_in_block':'Number of Employees','block_id':'CLUE Block Id'},
                           title='Employment Density & Venue Seating (2020)',
                           width=950, height=800
                          )

# Plot of venue seating
fig2 = px.scatter_mapbox(seatsByLocn, lat="y_coordinate", lon="x_coordinate", size="scale",
                        mapbox_style="stamen-toner",
                        zoom=12.15,
                        center = {"lat": -37.813, "lon": 144.945},
                        opacity=0.70,
                        hover_name="clue_area",
                        hover_data={"block_id":True,"scale":False,"number_of_seats":True,"x_coordinate":False,"y_coordinate":False},
                        color_discrete_sequence=['purple'],
                        labels={'number_of_seats':'Number of Seats', 'block_id':'CLUE Block Id'},
                        width=950, height=800)
fig.add_trace(fig2.data[0])
fig.update_geos(fitbounds="locations", visible=False)

fig.show()

<div class="usecase-section-header">Combining all map layers into one interactive map box visualisation</div>

Let's now build a single map box visualisation using our three datasets.

Our first step is to create a base plotly figure to which we can add each individual map plot as a new layer.

The title of the visualisation and any common parameters can be set using the fig.update_layout() method.

In the cell below we also have defined two custom colorscales, one continuous for the choropleth map and the other discrete for the scatter map plot.

We then create a figure for each dataset and add it as a layer to the base figure using the fig.add_trace() method.

In [ ]:
# Define custom colour scale for choropleth (continuous) and scatter (discrete)
custom_continuous_colorscale = [(0, "lightblue"), (0.25, "blue"), (1, "darkblue")]
custom_discrete_colorscale = ['red']

# Create the base figure to which layers(traces) will be added.
fig = go.Figure()

# Set the default style for the map
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(hovermode='closest')
fig.update_layout(mapbox_center_lat=-37.813, mapbox_center_lon=144.945, mapbox_zoom=12.15)
fig.update_layout(width=950, height=800)
fig.update_layout(title='Residential & Employment Density plus Venue Seating (2020)')
fig.update_layout(coloraxis_colorscale=custom_continuous_colorscale)
fig.update_layout(coloraxis_colorbar={'title':'Density'})

# Create the definition for the Residential Dwellings Layer
fig1 = px.choropleth_mapbox(dwellingsByBlock, geojson=block, locations='block_id', color='dwelling_count',
                           range_color=(0, dwellingsByBlock['dwelling_count'].max()),
                           featureidkey="properties.block_id",
                           hover_name='clue_area',
                           hover_data={'block_id':True,'dwelling_count':True},
                           labels={'dwelling_count':'Number of Dwellings','block_id':'CLUE Block Id'},
                           opacity=0.5,
                           
                          )
fig.add_trace(fig1.data[0]) # add this layer to the base figure

# Create the definition for the Employment Layer
fig2 = px.choropleth_mapbox(employmentByBlock, geojson=block, locations='block_id', color='total_employment_in_block',
                           range_color=(0, employmentByBlock['total_employment_in_block'].max()),
                           featureidkey="properties.block_id",
                           hover_name='clue_area',
                           hover_data={'block_id':True,'total_employment_in_block':True},
                           labels={'total_employment_in_block':'Number of Employees','block_id':'CLUE Block Id'},
                           opacity=0.5
                          )
fig.add_trace(fig2.data[0]) # add this layer to the base figure

# Create the definition for the Venue Seating Layer
fig3 = px.scatter_mapbox(seatsByLocn, lat="y_coordinate", lon="x_coordinate", size="scale",
                        hover_name="clue_area",
                        hover_data={"block_id":True,"scale":False,"number_of_seats":True,"x_coordinate":False,"y_coordinate":False},
                        labels={'number_of_seats':'Number of Seats', 'block_id':'CLUE Block Id'},
                        opacity=0.70, color_discrete_sequence=custom_discrete_colorscale
                        )
fig.add_trace(fig3.data[0]) # add this layer to the base figure

Finally, we define buttons and text to appear along the top of the map.

Each button turns on a combination of layers when it is clicked. The layers it turns on are defined in the 'visible' arg array with the order of boolean values corresponding to the map layers in the order they were added.

For example: When the 'Residential Density & Seating' button is clicked it turns on the 1st and 3rd layer as defined by the following argument 'visible':[True, False, True] . The 1st layer was the Residential Dwelling density choropleth map and the 3rd layer was the Venue Seating Scatter map.

In [ ]:
# Turn off all choropleth layers
fig.update_traces(visible=False, selector=dict(type='choroplethmapbox'))

# Add buttons for selection on plot
buttons = [dict(method='update',
                label='Venue Seating only',  visible=True,
                args=[{'label': 'Venue Seating', 'visible':[False, False, True]}]),
           dict(method='update',
                label='Residential Density & Seating', visible=True,
                args=[{'label': 'Residential Dwelling Density','visible':[True, False, True]}]),
           dict(method='update',
                label='Employment Density & Seating', visible=True,
                args=[{'label': 'Employment Density','visible':[False, True, True]}])
          ]
                   
um_buttons = [{'active':0, 'showactive':True, 'buttons':buttons,
               'direction': 'down', 'xanchor': 'left','yanchor': 'bottom', 'x': 0.71, 'y': 1.01}]
map_annotations = [{'text':'Please select a map view to display', 'x': 1, 'y': 1.1,
                    'showarrow': False, 'font':{'family':'Arial','size':14}}]

fig.update_layout(updatemenus=um_buttons, annotations=map_annotations)

# Display the map
fig.show()

<div class="usecase-section-header">Congratulations. Our interactive map is now complete!</div>

Now you can use the controls on the map above to explore the City of Melbourne and observe the residential density and employment density of each city block in relation to venue seating capacity.<br><br>

If you would like to extend this interactive map further, please visit the __[City of Melbourne Open Data Site](https://data.melbourne.vic.gov.au/)__ and explore some of the other valuable datasets including:
- __[Off Street Parking](https://data.melbourne.vic.gov.au/Transport/Off-street-car-parking-2020/g9am-cna5)__
- __[Pedestrian Counting System](https://data.melbourne.vic.gov.au/Transport/Pedestrian-Counting-System-Monthly-counts-per-hour/b2ak-trbp)__
- __[Microclimate sensor readings](https://data.melbourne.vic.gov.au/Environment/Microclimate-Sensor-Readings/u4vh-84j8?src=featured_banner)__
